In [1]:
import sqlite3
filename = '../../datasets/cbdata.sqlite'
conn = sqlite3.connect(filename)
cur = conn.cursor()
cur.executescript('''
CREATE TABLE IF NOT EXISTS Publications (id TEXT NOT NULL PRIMARY KEY UNIQUE, title TEXT);
CREATE TABLE IF NOT EXISTS Datasets (id TEXT NOT NULL PRIMARY KEY UNIQUE, title TEXT);
CREATE TABLE IF NOT EXISTS Keywords (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, keyword TEXT);
CREATE TABLE IF NOT EXISTS Referencing (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, publication_id INTEGER, dataset_id TEXT, keyword_id INTEGER);
''')
conn.commit()

In [2]:
import pandas as pd
csvname = '../../datasets/osdr_references.csv'
df = pd.read_csv(csvname)

In [3]:
for index, data in df.iterrows():
  data = data.fillna('')
  dataset_title = ''
  publication_title = data['paper_title']
  publication_id = data['paper_pmcid']
  dataset_id = data['dataset_accession']

  cur.execute('SELECT COUNT(*) FROM Datasets WHERE id=(?)', (dataset_id,))
  count = cur.fetchone()[0]
  if count == 0:
    cur.execute('INSERT INTO Datasets (id, title) VALUES (?, ?)', (dataset_id, dataset_title))
  cur.execute('SELECT COUNT(*) FROM Publications WHERE id=(?)', (publication_id,))
  count = cur.fetchone()[0]
  if count == 0:
    cur.execute('INSERT INTO Publications (id, title) VALUES (?, ?)', (publication_id, publication_title))

  mission = data['mined_mission_terms'].split(';')
  for keyword in mission:
    keyword = keyword.lower()
    if len(keyword) <= 1:
      continue
    cur.execute('SELECT COUNT(*) FROM Keywords WHERE keyword=(?)', (keyword,))
    count = cur.fetchone()[0]
    if count == 0:
      cur.execute('INSERT INTO Keywords (keyword) VALUES (?)', (keyword,))
    cur.execute('SELECT id FROM Keywords WHERE keyword=(?)', (keyword,))
    keyword_id = cur.fetchone()[0]
    cur.execute('INSERT INTO Referencing (publication_id, dataset_id, keyword_id) VALUES (?, ?, ?)', (publication_id, dataset_id, keyword_id))
  
  organism = data['mined_organism_terms'].split(';')
  for keyword in organism:
    keyword = keyword.lower()
    if len(keyword) <= 1:
      continue
    cur.execute('SELECT COUNT(*) FROM Keywords WHERE keyword=(?)', (keyword,))
    count = cur.fetchone()[0]
    if count == 0:
      cur.execute('INSERT INTO Keywords (keyword) VALUES (?)', (keyword,))
    cur.execute('SELECT id FROM Keywords WHERE keyword=(?)', (keyword,))
    keyword_id = cur.fetchone()[0]
    cur.execute('INSERT INTO Referencing (publication_id, dataset_id, keyword_id) VALUES (?, ?, ?)', (publication_id, dataset_id, keyword_id))
  
  assay = data['mined_assay_terms'].split(';')
  for keyword in assay:
    keyword = keyword.lower()
    if len(keyword) <= 1:
      continue
    cur.execute('SELECT COUNT(*) FROM Keywords WHERE keyword=(?)', (keyword,))
    count = cur.fetchone()[0]
    if count == 0:
      cur.execute('INSERT INTO Keywords (keyword) VALUES (?)', (keyword,))
    cur.execute('SELECT id FROM Keywords WHERE keyword=(?)', (keyword,))
    keyword_id = cur.fetchone()[0]
    cur.execute('INSERT INTO Referencing (publication_id, dataset_id, keyword_id) VALUES (?, ?, ?)', (publication_id, dataset_id, keyword_id))

  cur.execute('INSERT INTO Referencing (publication_id, dataset_id) VALUES (?, ?)', (publication_id, dataset_id))
conn.commit()